In [ ]:
# STEGANALOG: Trainer - naukowo zgodna wersja (zoptymalizowana pod kątem prędkości)
# Optymalizacje: Warstwa SRM, szybsza obsługa JPEG Augmentation (w pamięci), więcej workerów DataLoader.

import os
from pathlib import Path
import random
import shutil
import math
import json
import time
import io # Dodane dla szybkiego I/O w pamięci (JPEG Augmentation)

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL.PngImagePlugin import PngInfo
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision import transforms, datasets
import torchvision.transforms.functional as TF

from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score, confusion_matrix, roc_curve, auc, precision_recall_curve, accuracy_score
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm

# ============================
# ========== USTAWIENIA ======
# ============================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# Ustawienie liczby workerów na podstawie dostępnych rdzeni CPU (maks. 4 w Colab, by uniknąć problemów)
# Zmienione z 2 na 4 dla lepszej wydajności wczytywania
NUM_WORKERS = 4 if DEVICE == "cuda" else 0
print("Urządzenie:", DEVICE)

# Ścieżki — dostosuj do własnego Drive
DATA_DIR = Path("/content/drive/MyDrive/stegano_dataset")
OUTPUT_DIR = Path("/content/drive/MyDrive/stegano_dataset/results_sci_optimized")
MY_SOURCE_FOLDER = Path("/content/drive/MyDrive/MY_IPHONE_DATASET") # Opcjonalnie (już nie używane)

for p in (DATA_DIR, OUTPUT_DIR):
    p.mkdir(parents=True, exist_ok=True)
print("Data:", DATA_DIR, "Output:", OUTPUT_DIR)

BATCH_SIZE = 16
NUM_CLASSES = 2
EPOCHS = 12
LR = 1e-4
WEIGHT_DECAY = 1e-4
IMG_SIZE = 256 # rozmiar wejściowy (zachowany 256)
USE_MIXUP = True
USE_JPEG_AUG = True  # podczas treningu losowe QF (imitacja post-processingu)
SVM_C = 3.0

# ============================
# ======= STEG GENERATOR =====
# (Funkcje pomocnicze, nie używane w głównym treningu)
# ============================
def hide_lsb_image(img: Image.Image, message: str = "secret"):
    # proste LSB w kanale R
    img = img.convert("RGB")
    arr = np.array(img, dtype=np.uint8)
    bin_msg = ''.join(format(ord(c), '08b') for c in message)
    flat = arr[:, :, 0].flatten()
    n = min(len(bin_msg), flat.size)
    for i in range(n):
        flat[i] = (flat[i] & ~1) | int(bin_msg[i])
    arr[:, :, 0] = flat.reshape(arr.shape[0], arr.shape[1])
    return Image.fromarray(arr)

def hide_dct_jpeg(in_path: str, out_path: str, quality: int = 40):
    # zapisz jako JPEG z niską jakością (symulacja DCT-based steg)
    img = cv2.imread(in_path)
    if img is None:
        return False
    cv2.imwrite(str(out_path), img, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
    return True

def hide_meta_png(src_path: str, out_path: str, comment: str = "stegano_demo"):
    img = Image.open(src_path)
    pnginfo = PngInfo()
    pnginfo.add_text("comment", comment)
    img.save(out_path, "PNG", pnginfo=pnginfo)

def generate_stegano_from_file(src_path: Path, dest_path: Path):
    method = random.choice(["lsb", "dct", "meta"])
    if method == "lsb":
        img = Image.open(src_path)
        steg = hide_lsb_image(img, message="HiddenByUser")
        steg.save(dest_path)
    elif method == "dct":
        # użyj tymczasowego pliku
        hide_dct_jpeg(str(src_path), str(dest_path), quality=random.randint(30,60))
    else:
        hide_meta_png(str(src_path), str(dest_path))
    return method


# ============================
# ===== DATALOADER I TRANSFORMS =
# ============================
# Transformacje obrazu (augmentacje)
train_tf_img = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.05,0.05,0.05,0.02),
])

val_tf_img = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
])

# Normalizacja (ToTensor + Normalize)
to_tensor_norm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# Kompletne pipeline transformacji
train_transform = transforms.Compose([train_tf_img, to_tensor_norm])
val_transform = transforms.Compose([val_tf_img, to_tensor_norm])


# Dataset dostosowany do nowej struktury katalogów i aug. JPEG
class SteganoDataset(torch.utils.data.Dataset):
    def __init__(self, samples, transform=None, jpeg_q_aug=False):
        self.samples = samples
        self.transform = transform
        self.jpeg_q_aug = jpeg_q_aug

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert("RGB")

        # 🔥 OPTYMALIZACJA: Szybszy JPEG Augmentation w pamięci (bez I/O na dysku)
        if self.jpeg_q_aug and random.random() < 0.5:
            q = random.randint(30,95)
            # Użycie io.BytesIO do zapisu/odczytu w pamięci
            buffer = io.BytesIO()
            img.save(buffer, "JPEG", quality=q)
            buffer.seek(0)
            img = Image.open(buffer).convert("RGB") # Wczytaj ponownie z bufora
            buffer.close()

        if self.transform:
            img = self.transform(img)

        return img, label

# ==============================================
# 📂 WCZYTYWANIE I BALANSOWANIE DANYCH
# ==============================================
# Sprawdzenie istnienia folderów
cover_dir = DATA_DIR / "cover"
pseudo_dir = DATA_DIR / "pseudo"
stego_dir = DATA_DIR / "stego"

if not (cover_dir.exists() and pseudo_dir.exists() and stego_dir.exists()):
    raise FileNotFoundError(f"Brak wymaganych folderów (cover/pseudo/stego) w {DATA_DIR}")

# 🔥 Wczytanie ścieżek
cover_files = list(cover_dir.rglob("*.*"))
pseudo_files = list(pseudo_dir.rglob("*.*"))
stego_files = list(stego_dir.rglob("*.*"))

# Odsianie plików, które nie są obrazami
def filter_images(files):
    return [p for p in files if p.suffix.lower() in (".jpg", ".jpeg", ".png", ".bmp")]

cover_files = filter_images(cover_files)
pseudo_files = filter_images(pseudo_files)
stego_files = filter_images(stego_files)

# ⚖️ BALANSOWANIE DANYCH
zero_class = cover_files + pseudo_files # label 0
one_class = stego_files # label 1

print(f"ZERO (cover+pseudo): {len(zero_class)}")
print(f"ONE (stego): {len(one_class)}")

# Maksymalna liczba próbek 0 = 2 × liczba stego (delikatne balansowanie)
max_zero = min(len(zero_class), len(one_class) * 2)

random.shuffle(zero_class)
zero_class = zero_class[:max_zero]

print(f"ZERO (after balancing): {len(zero_class)}")
print(f"Total samples: {len(zero_class) + len(one_class)}")

# 🔧 Tworzymy listę (plik, label)
all_samples = [(str(p), 0) for p in zero_class] + [(str(p), 1) for p in one_class]
random.shuffle(all_samples)


# ✂️ Split: 70/15/15 ze stratyfikacją
# Ponieważ 'all_samples' jest listą ścieżek, musimy najpierw wydobyć etykiety
labels = [s[1] for s in all_samples]
indices = list(range(len(all_samples)))

train_idx, temp_idx = train_test_split(indices, test_size=0.30, stratify=labels, random_state=SEED)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, stratify=np.array(labels)[temp_idx], random_state=SEED)

train_samples = [all_samples[i] for i in train_idx]
val_samples = [all_samples[i] for i in val_idx]
test_samples = [all_samples[i] for i in test_idx]

# Tworzenie datasetów
train_ds = SteganoDataset(train_samples, transform=train_transform, jpeg_q_aug=USE_JPEG_AUG)
val_ds = SteganoDataset(val_samples, transform=val_transform, jpeg_q_aug=False)
test_ds = SteganoDataset(test_samples, transform=val_transform, jpeg_q_aug=False)

# 🔥 Użycie NUM_WORKERS dla lepszej wydajności wczytywania danych
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print("Split sizes: train", len(train_ds), "val", len(val_ds), "test", len(test_ds))

# ============================
# ===== High-pass (SRM) LAYER =
# ============================
class SRMLayer(nn.Module):
    """
    Warstwa z kilkoma stałymi filtrami górnoprzepustowymi (uproszczenie SRM).
    Zoptymalizowana do użycia jednej konwolucji grupowanej, co jest szybsze.
    """
    def __init__(self, in_channels=3):
        super().__init__()
        # Przykładowe filtry (3x3 / 5x5) - uproszczone SRM-like
        kv = np.array([[0, -1, 0],
                       [-1, 4, -1],
                       [0, -1, 0]], dtype=np.float32) # laplacian-like
        kv3 = np.array([[-1, 2, -1],
                         [2, -4, 2],
                         [-1, 2, -1]], dtype=np.float32) # Drugi filtr

        kernels = [kv, kv3]

        # 🔥 OPTYMALIZACJA: Tworzenie pojedynczego tensora wag dla konwolucji grupowej
        conv_kernels_list = []
        for k in kernels:
            k = torch.from_numpy(k).float().unsqueeze(0).unsqueeze(0) # 1x1xHxW
            conv_kernels_list.append(k)

        # Skonkatenuj filtry i powtórz dla każdego kanału wejściowego
        # final_weight shape: (C * n_kernels, 1, kH, kW)
        weight_per_channel = torch.cat(conv_kernels_list, dim=0) # n_kernels x 1 x kH x kW

        # Powtórz dla każdego kanału wejściowego (depthwise style: in_ch=1)
        final_weight = weight_per_channel.repeat(in_channels, 1, 1, 1)

        self.register_buffer('kernels', final_weight)
        self.in_channels = in_channels
        self.n_kernels = len(kernels)
        self.out_channels = in_channels * self.n_kernels
        self.kernel_size = kernels[0].shape[0]

    def forward(self, x):
        # x: B x C x H x W
        weight = self.kernels.to(x.device)
        padding = self.kernel_size // 2

        # 🔥 OPTYMALIZACJA: Użycie jednej konwolucji grupowej (depthwise)
        # groups=C - dla każdej grupy (kanału wejściowego) stosowane jest n_kernels filtrów
        out = F.conv2d(x, weight, padding=padding, groups=self.in_channels)

        return torch.tanh(out)

# ============================
# ======= SRNet-like CNN ======
# ============================
# Prosty residual-focused CNN (inspiracja SRNet/Yedroudj) - pracuje na residualu (wyjściu SRMLayer)
class SimpleStegNet(nn.Module):
    def __init__(self, in_channels=6, num_classes=2, feature_dim=512):
        """
        in_channels: liczba kanałów wejściowych (np. C * n_kernels)
        """
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        self.res_block1 = self._make_res_block(64, 128)
        self.res_block2 = self._make_res_block(128, 256)
        self.res_block3 = self._make_res_block(256, 512)
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, feature_dim)
        self.classifier = nn.Linear(feature_dim, num_classes)

        # Zmiana: użycie MaxPool zamiast AvgPool na końcu bloku dla lepszej detekcji lokalnych cech
        self.res_block1 = self._make_res_block(64, 128)
        self.res_block2 = self._make_res_block(128, 256)
        self.res_block3 = self._make_res_block(256, 512)

    def _make_res_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2) # Zmiana z AvgPool2d na MaxPool2d - może dać lepsze wyniki
        )

    def forward(self, x):
        # x expected residual map (B x in_channels x H x W)
        x = self.conv1(x)
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.pool(x).view(x.size(0), -1)
        feats = self.fc(x)
        logits = self.classifier(feats)
        return logits, feats

# Pełny model: SRM layer + SRNet
class FullStegModel(nn.Module):
    def __init__(self, in_rgb_channels=3, num_classes=2, feature_dim=512):
        super().__init__()
        self.srm = SRMLayer(in_channels=in_rgb_channels)
        # SRMLayer produces C * nk channels; nk=2 here => 3*2=6
        self.net = SimpleStegNet(in_channels=in_rgb_channels * 2, num_classes=num_classes, feature_dim=feature_dim)

    def forward(self, x):
        # input x: B x 3 x H x W (normalized floats)
        residual = self.srm(x) # B x (C*nk) x H x W
        logits, feats = self.net(residual)
        return logits, feats

# ============================
# ====== UTILITY: mixup ======
# ============================
def mixup_data(x, y, alpha=0.4):
    if alpha <= 0:
        return x, y, None, None, 1.0
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(x.size(0)).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, index, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# ============================
# ===== TRAIN / EVAL (PŁEĆ) =
# ============================
model = FullStegModel(in_rgb_channels=3, num_classes=NUM_CLASSES, feature_dim=512).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Zmniejszono cierpliwość dla szybszego uczenia
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1)
criterion = nn.CrossEntropyLoss()

def train_one_epoch(model, loader, optimizer, criterion, use_mixup=False):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    # Dodano torch.autocast dla operacji na GPU (FP16 dla przyspieszenia)
    scaler = torch.cuda.amp.GradScaler() if DEVICE == "cuda" else None

    for imgs, labels in tqdm(loader, desc="Train", leave=False):
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        if DEVICE == "cuda":
            with torch.cuda.amp.autocast():
                if use_mixup:
                    imgs_m, y_a, y_b, idx, lam = mixup_data(imgs, labels)
                    outputs, _ = model(imgs_m)
                    loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
                else:
                    outputs, _ = model(imgs)
                    loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else: # CPU Path
            if use_mixup:
                imgs_m, y_a, y_b, idx, lam = mixup_data(imgs, labels)
                outputs, _ = model(imgs_m)
                loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
            else:
                outputs, _ = model(imgs)
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(1)
        # Liczenie poprawnych dla celów logowania
        if use_mixup:
            # W przypadku mixup dokładność jest mniej wiarygodna, ale liczymy ją w uproszczeniu
            correct += (preds == labels).sum().item()
        else:
            correct += (preds == labels).sum().item()
        total += labels.size(0)
    return total_loss / len(loader), correct / total

def evaluate_model(model, loader, criterion):
    """ Uruchamia inferencję i zbiera wyniki (etykiety, predykcje, prawdopodobieństwa) """
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_labels = []
    probs = []
    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc="Eval", leave=False):
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)

            if DEVICE == "cuda":
                with torch.cuda.amp.autocast():
                    outputs, _ = model(imgs)
                    loss = criterion(outputs, labels)
            else:
                outputs, _ = model(imgs)
                loss = criterion(outputs, labels)

            total_loss += loss.item()
            # Prawdopodobieństwo klasy 1 (stego)
            p = F.softmax(outputs, dim=1)[:,1]
            preds = outputs.argmax(1)

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())
            probs.extend(p.cpu().tolist())

    y_pred = np.array(all_preds)
    y_true = np.array(all_labels)
    y_probs = np.array(probs)

    # Używamy predykcji opartych na progu 0.5 dla wstępnej oceny
    # Uwaga: dla końcowej metryki AUC/Accuracy to nie ma znaczenia, ale dla CM i F1 ma
    return total_loss / len(loader), y_pred, y_true, y_probs

def calculate_and_save_metrics(y_true, y_pred, y_probs, set_name, epoch_num, out_dir, threshold=0.5):
    """ Oblicza i zapisuje pełen zestaw metryk ewaluacyjnych. """
    # Używamy progu do binarnej klasyfikacji
    y_pred_thr = (y_probs >= threshold).astype(int)

    # Ustawienie katalogu wyjściowego
    # Jeśli epoch_num jest None (ocena końcowa), użyj nazwy zbioru
    if epoch_num is not None:
        epoch_dir = out_dir / f"epoch_{epoch_num:02d}_{set_name}_metrics"
    else:
        epoch_dir = out_dir / f"final_{set_name}_metrics"

    epoch_dir.mkdir(parents=True, exist_ok=True)

    # 1. Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_thr)
    plt.figure(figsize=(5,5))
    plt.imshow(cm, cmap='Blues')
    title = f"Confusion Matrix ({set_name}, Epoch {epoch_num or 'Final'})"
    plt.title(title)
    plt.xlabel("Pred"); plt.ylabel("True")
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j,i, str(cm[i,j]), ha='center', va='center',
                     color='white' if cm[i,j]>cm.max()/2 else 'black')
    plt.colorbar()
    plt.savefig(epoch_dir / "confusion_matrix.png"); plt.close()

    # 2. ROC and AUC
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(6,6))
    plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
    plt.plot([0,1],[0,1],'k--')
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC Curve ({set_name}, Epoch {epoch_num or 'Final'})")
    plt.legend(); plt.grid(True)
    plt.savefig(epoch_dir / "roc_curve.png"); plt.close()

    # 3. Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_true, y_probs)
    plt.figure(figsize=(6,6))
    plt.plot(recall, precision)
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"Precision-Recall ({set_name}, Epoch {epoch_num or 'Final'})")
    plt.grid(True)
    plt.savefig(epoch_dir / "pr_curve.png"); plt.close()

    # 4. Classification Report (JSON)
    report = classification_report(y_true, y_pred_thr, output_dict=True)
    with open(epoch_dir / "classification_report.json", "w", encoding="utf-8") as f:
        json.dump(report, f, indent=4, ensure_ascii=False)

    accuracy = accuracy_score(y_true, y_pred_thr)
    return accuracy, roc_auc, report

# ============================
# ===== MAIN TRAINING LOOP ===
# ============================
train_hist = {"loss": [], "acc": [], "val_loss": [], "val_acc": [], "val_auc": []}
best_val_acc = 0.0
best_epoch = -1

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    t0 = time.time()
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, use_mixup=USE_MIXUP)

    # === EWENTRUALNE ZAPISYWANIE STATYSTYK WALIDACYJNYCH ===
    val_loss, _, val_y_true, val_y_probs = evaluate_model(model, val_loader, criterion)

    # ❗️ Używamy progu 0.5 dla spójnej metryki walidacyjnej w trakcie treningu
    val_acc, val_auc, _ = calculate_and_save_metrics(
        val_y_true, None, val_y_probs,
        set_name="val", epoch_num=epoch+1, out_dir=OUTPUT_DIR, threshold=0.5
    )

    scheduler.step(val_acc)
    dt = time.time() - t0

    # ❗️ WYŚWIETLENIE ACCURACY I AUC
    print(f"TrainLoss={train_loss:.4f} TrainAcc={train_acc:.4f} | ValLoss={val_loss:.4f} ValAcc={val_acc:.4f} ValAUC={val_auc:.4f} | time={dt:.1f}s")

    train_hist["loss"].append(train_loss)
    train_hist["acc"].append(train_acc)
    train_hist["val_loss"].append(val_loss)
    train_hist["val_acc"].append(val_acc)
    train_hist["val_auc"].append(val_auc) # Zapisujemy AUC

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch
        torch.save(model.state_dict(), OUTPUT_DIR / "best_stegnet.pth")
        print("✅ Zapisano najlepsze wagi (CNN).")

print(f"\nNajlepszy ValAcc: {best_val_acc:.4f} (ep: {best_epoch+1})")
# wczytaj najlepsze wagi
try:
    model.load_state_dict(torch.load(OUTPUT_DIR / "best_stegnet.pth", map_location=DEVICE))
except FileNotFoundError:
    print("Błąd: Nie znaleziono najlepszych wag. Kontynuacja z ostatnimi wagami.")


# ============================
# ===== EKSTRAKCJA CECH (TEST) =
# ============================
def extract_features_from_loader(model, loader):
    model.eval()
    feats, labels = [], []
    with torch.no_grad():
        for imgs, lbls in tqdm(loader, desc="Extract"):
            imgs = imgs.to(DEVICE)

            if DEVICE == "cuda":
                with torch.cuda.amp.autocast(): # Użycie autocast do ekstrakcji cech
                    _, f = model(imgs)
            else:
                _, f = model(imgs)

            feats.append(f.cpu().numpy())
            labels.append(lbls.numpy())
    feats = np.vstack(feats)
    labels = np.concatenate(labels)
    return feats, labels

print("\n--- Rozpoczęcie końcowej ewaluacji (CNN + SVM Ensemble) ---")
train_feats, train_lbls = extract_features_from_loader(model, train_loader)
test_feats, test_lbls = extract_features_from_loader(model, test_loader)
print("Feats shapes:", train_feats.shape, test_feats.shape)

# ============================
# ===== SVM classifier (TEST) =======
# ============================
svm = SVC(kernel='rbf', probability=True, C=SVM_C, gamma='scale', random_state=SEED)
svm.fit(train_feats, train_lbls)
svm_probs = svm.predict_proba(test_feats)[:,1]

# also CNN probs
def cnn_predict_probs(model, loader):
    model.eval()
    probs, labels = [], []
    with torch.no_grad():
        for imgs, lbls in tqdm(loader, desc="CNN predict"):
            imgs = imgs.to(DEVICE)

            if DEVICE == "cuda":
                with torch.cuda.amp.autocast():
                    outputs, _ = model(imgs)
            else:
                outputs, _ = model(imgs)

            p = F.softmax(outputs, dim=1)[:,1]
            probs.append(p.cpu().numpy())
            labels.append(lbls.numpy())
    return np.concatenate(probs), np.concatenate(labels)

cnn_probs, y_true = cnn_predict_probs(model, test_loader)

# simple ensemble: average prob
ensemble_probs = (cnn_probs + svm_probs) / 2.0

# best threshold by F1
best_thr, best_f1 = 0.5, 0.0
for thr in np.linspace(0.1,0.9,81):
    preds = (ensemble_probs >= thr).astype(int)
    f1 = f1_score(y_true, preds)
    if f1 > best_f1:
        best_f1, best_thr = f1, thr
print(f"Najlepszy próg (na zbiorze testowym): {best_thr:.3f}, F1: {best_f1:.3f}")

final_preds = (ensemble_probs >= best_thr).astype(int)

# save models
joblib.dump(svm, OUTPUT_DIR / "svm_stegano.pkl")
torch.save(model.state_dict(), OUTPUT_DIR / "best_stegnet_final.pth")
print("✅ Zapisano modele w:", OUTPUT_DIR)

# ============================
# ===== WIZUALIZACJE KOŃCOWE (TEST) =========
# ============================
def save_plot_history(hist, out_dir):
    out_dir.mkdir(parents=True, exist_ok=True)

    plt.figure(figsize=(8,4))
    plt.plot(hist['loss'], label='Train loss')
    plt.plot(hist['val_loss'], label='Val loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
    plt.legend(); plt.grid(True)
    plt.savefig(out_dir / "loss_history.png"); plt.close()

    plt.figure(figsize=(8,4))
    plt.plot(hist['acc'], label='Train acc')
    plt.plot(hist['val_acc'], label='Val acc')
    plt.plot(hist['val_auc'], label='Val AUC') # Dodano AUC do historii
    plt.xlabel('Epoch'); plt.ylabel('Metric Value'); plt.title('Accuracy and AUC History')
    plt.legend(); plt.grid(True)
    plt.savefig(out_dir / "acc_auc_history.png"); plt.close()

save_plot_history(train_hist, OUTPUT_DIR)

# Zapisanie końcowych metryk na zbiorze testowym (Ensemble)
final_test_acc, final_test_auc, final_report = calculate_and_save_metrics(
    y_true, final_preds, ensemble_probs,
    set_name="test_ensemble", epoch_num=None, out_dir=OUTPUT_DIR, threshold=best_thr
)

print("--- KOŃCOWY RAPORT EWALUACYJNY (TEST SET, ENSEMBLE) ---")
print(f"Accuracy: {final_test_acc:.4f}, AUC: {final_test_auc:.4f}, Threshold: {best_thr:.3f}")
print("Wszystkie statystyki zapisano w:", OUTPUT_DIR / "final_test_ensemble_metrics")

# ============================
# ====== PRZYKŁADOWE PREDYKCJE =
# ============================
def show_examples(model, svm, dataset_subset, best_thr, n_examples=8, out_dir=OUTPUT_DIR):
    model.eval()
    idxs = random.sample(range(len(dataset_subset)), min(n_examples, len(dataset_subset)))
    fig, axes = plt.subplots(2, math.ceil(n_examples/2), figsize=(16,8))
    axes = axes.flatten()
    for i, ix in enumerate(idxs):
        img_tensor, label = dataset_subset[ix]
        # img_tensor is normalized; undo normalization for display
        img = img_tensor.cpu().numpy().transpose(1,2,0)
        mean = np.array([0.485,0.456,0.406]); std = np.array([0.229,0.224,0.225])
        img = np.clip(img * std + mean, 0, 1)
        axes[i].imshow(img)
        axes[i].axis('off')
        t = img_tensor.unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            if DEVICE == "cuda":
                with torch.cuda.amp.autocast():
                    out, feats = model(t)
            else:
                out, feats = model(t)

            cnn_p = float(F.softmax(out, dim=1)[:,1].cpu().numpy()[0])
        svm_p = float(svm.predict_proba(feats.cpu().numpy())[:,1][0])
        ensemble_p = (cnn_p + svm_p) / 2.0
        pred_label = "stegano" if ensemble_p >= best_thr else "cover"
        true_label = "stegano" if label==1 else "cover"
        axes[i].set_title(f"Prawda:{true_label}\nPred:{pred_label} ({ensemble_p:.2f})\nCNN:{cnn_p:.2f} SVM:{svm_p:.2f}", fontsize=9)
    plt.tight_layout()
    out_dir.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_dir / "examples_predictions.png")
    plt.close()

show_examples(model, svm, test_ds, best_thr=best_thr, n_examples=8, out_dir=OUTPUT_DIR)
print("✅ Zapisano przykładowe predykcje.")
print("Koniec. Wyniki zapisane w folderze:", OUTPUT_DIR)
# ============================
# ====== KONIEC PLIKU =======
# ============================